In [1]:
#!/usr/bin/env python3
import ete3
import Bio.AlignIO
import Bio.Seq
import Bio.Align
import numpy
import os

In [ ]:
# get amino acid alignment
aln_cds = Bio.AlignIO.read('../../0_data/selectionAndASR/5133_NT_AL.fasta', format='fasta')
print(aln_cds)
with open('5133_NT_AL.aa.fasta', 'w') as f:
    for seq_record in aln_cds:
        print(f'>{seq_record.id}', file=f)
        print(f'{seq_record.seq.translate()}', file=f)
aln_aa = Bio.AlignIO.read('5133_NT_AL.aa.fasta', format='fasta')
print(aln_aa)

sp_list = [x.id for x in aln_aa]

In [ ]:
# import tree
tree = ete3.Tree('../../0_data/selectionAndASR/timetree_all_sp.treefile')
tree_sps = tree.get_leaf_names()

# remove gap-rich sequences
out_dir = '../03_orthomam_filterseq10/'
os.makedirs(out_dir, exist_ok=True)
# aln_cds = Bio.AlignIO.read('5133_NT_AL.fasta', format='fasta')
total_length = aln_cds.get_alignment_length()
filtered_sp_list = []
filtered_seqrec_list = []
for seq_record in aln_cds:
    if str(seq_record.seq).count('-') > 0.1 * total_length:
        continue
    else:
        filtered_sp_list.append(seq_record.id)
        filtered_seqrec_list.append(seq_record)
filtered_aln_cds = Bio.Align.MultipleSeqAlignment(filtered_seqrec_list)
filtered_aln_cds_arr = numpy.array(filtered_aln_cds)
nongap_indices = [x for x in range(filtered_aln_cds_arr.shape[1])
                  if not numpy.all(filtered_aln_cds_arr[:, x] == '-')]
filtered_aln_cds_arr = filtered_aln_cds_arr[:, nongap_indices]
for i in range(len(filtered_aln_cds)):
    filtered_aln_cds[i].seq = Bio.Seq.Seq(''.join(filtered_aln_cds_arr[i]))
print(filtered_aln_cds)
need_sps = list(set(filtered_sp_list)&set(tree_sps))
with open(f'{out_dir}/5133_NT_AL.filtered10.fasta', 'w') as f:
    for seq_record in filtered_aln_cds:
        if seq_record.id in need_sps:
            print(f'>{seq_record.id}', file=f)
            print(f'{seq_record.seq}', file=f)

with open(f'{out_dir}/5133_NT_AL.filtered10.aa.fasta', 'w') as f:
    for seq_record in filtered_aln_cds:
        if seq_record.id in need_sps:
            print(f'>{seq_record.id}', file=f)
            print(f'{seq_record.seq.translate()}', file=f)
aln_aa = Bio.AlignIO.read(f'{out_dir}5133_NT_AL.filtered10.aa.fasta', format='fasta')

# get filtered species tree
tree.prune(need_sps, preserve_branch_length=True)
print(len(tree.get_leaf_names()))
with open(f'{out_dir}/timetree.treefile', 'w') as f:
    print(tree.write(format=9), file=f)

In [ ]:
# make selection results dictories
def mkdir(path):
	folder = os.path.exists(path)
	if not folder:                  
		os.makedirs(path, exist_ok=True)            
		
file = f'{out_dir}/' 
splist = ['primate','rodent'] 
dirlist = ['busted',  'relax', 'busted_within_clade', 'relax_within_clade'] 
for sp in splist:
    for dir in dirlist:
        mkdir(os.path.join(file, sp, dir))

In [13]:
# build selection tree__primate
out_dir_s = f'{out_dir}/rodent'

# MRCA
tree = ete3.Tree(f'{out_dir}/timetree.treefile')
selection_node = tree.get_common_ancestor([tree & 'Homo_sapiens', tree & 'Lemur_catta'])
selection_node.name = str(selection_node.name) + '{Foreground}'
with open(f'{out_dir_s}/busted/timetree_filtered10_fg.nw', 'w') as f:
    print(tree.write(format=1), file=f)
tree.write(format=1, outfile=f'{out_dir_s}/relax/timetree_filtered10_fg.nw')

# within_clade
tree = ete3.Tree(f'{out_dir}/timetree.treefile')
selection_node = tree.get_common_ancestor([tree & 'Homo_sapiens', tree & 'Lemur_catta'])
for node in selection_node.iter_descendants("postorder"):
    node.name = str(node.name)+'{Foreground}'
tree.write(format=1, outfile=f'{out_dir_s}/relax_within_clade/timetree_filtered10_fg_inclade_nosp.nw')
tree.write(format=1, outfile=f'{out_dir_s}/busted_within_clade/timetree_filtered10_fg_inclade_nosp.nw')

In [4]:
# build selection tree__rodent
out_dir_s = f'{out_dir}/rodent'

# MRCA
tree = ete3.Tree(f'{out_dir}/timetree.treefile')
selection_node = tree.get_common_ancestor([tree & 'Mus_musculus', tree & 'Marmota_monax'])
selection_node.name = str(selection_node.name) + '{Foreground}'
with open(f'{out_dir_s}/busted/timetree_filtered10_fg.nw', 'w') as f:
    print(tree.write(format=1), file=f)
tree.write(format=1, outfile=f'{out_dir_s}/relax/timetree_filtered10_fg.nw')

# within_clade
tree = ete3.Tree(f'{out_dir}/timetree.treefile')
selection_node = tree.get_common_ancestor([tree & 'Mus_musculus', tree & 'Marmota_monax'])
for node in selection_node.iter_descendants("postorder"):
    node.name = str(node.name)+'{Foreground}'
tree.write(format=1, outfile=f'{out_dir_s}/relax_within_clade/timetree_filtered10_fg_inclade_nosp.nw')
tree.write(format=1, outfile=f'{out_dir_s}/busted_within_clade/timetree_filtered10_fg_inclade_nosp.nw')

In [ ]:
import os
dirs = []
for dir in os.listdir(out_dir):
    if dir.endswith("10.fasta"):
        fa_file = os.path.join(out_dir, dir)
    if "." not in dir:
        dirs.append(dir)

for dir in dirs:
    for root, dirss, files in os.walk(os.path.join(out_dir, dir)):
        for subdir in dirss:
            json_files = [file for file in os.listdir(os.path.join(root, subdir)) if file.endswith('.json')]
            if not json_files:
                tree_file = [file for file in os.listdir(os.path.join(root, subdir)) if file.endswith('.nw')]
                tree_file = os.path.join(root, subdir, tree_file[0])
                if 'busted' in subdir:
                    command = f'hyphy CPU=1 busted --alignment {fa_file} --branches Foreground --tree {tree_file} --output {root}/{subdir}/{subdir}_{fa_file.split("/")[-1].replace(".fasta",".json")} > {root}/{subdir}/{subdir}_{fa_file.split("/")[-1].replace(".fasta",".out")} &'
                elif 'relax' in subdir:
                    command = f'hyphy CPU=1 relax --alignment {fa_file} --test Foreground --tree {tree_file} --output {root}/{subdir}/{subdir}_{fa_file.split("/")[-1].replace(".fasta",".json")} > {root}/{subdir}/{subdir}_{fa_file.split("/")[-1].replace(".fasta",".out")} &'
                else:
                    command = ''
                os.system(command)
                # print(command)